# Compound Target

This example demonstrates how to simulate the passage of a collinear beam of photons through a target that consists of several layers.
The case study is a nuclear resonance fluorescence (NRF) experiment whose goal is to measure the width of the $2^+_1$ state of the isotope $^{116}$Sn relative to the known width of the $2^+_1$ state of $^{112}$Sn.

The formalism in this notebook is based on the nice overview in C. Romig's thesis:

C. Romig, Dissertation, Technische Universität Darmstadt, Darmstadt (2015) https://tuprints.ulb.tu-darmstadt.de/id/eprint/4446

This text, in turn, is based on the diploma thesis of N. Pietralla (which is not available publicly in digital form, unfortunately):

N. Pietralla, Diplomarbeit, Universität zu Köln, Köln (1993)

## Target

The target consists of 4 stacked layers that contain enriched metallic tin material at standard conditions.
They will be numbered starting with the layer that is hit first by the beam. 
The first ($^{116}$Sn-1) and the third ($^{116}$Sn-2) layer have a high enrichment $x(^{116}\mathrm{Sn})$ in the isotope $^{116}$Sn, while the second ($^{112}$Sn-1) and fourth ($^{112}$Sn-2) layer mainly consist of $^{112}$Sn.
All targets are cylindrical with a radius of 0.5 cm.
Using the target cross section area, the areal particle density (number of target nuclei per femtometer squared) is calculated below for compatibility with the cross sections, which are given in femtometers squared.

In the code, $^{112}$Sn and $^{116}$Sn are represented by two `Isotope` objects that contain information about their masses, and their ground- and excited states.
States are represented by `GroundState` and `State` objects.
The module `ries.constituents.element` provides natural-element properties, particular isotope masses, and the code below shows how to access them.
No compilation like this exists for the states of all isotopes, so the data entered below were looked up by hand in the Evaluated Nuclear Structure Data File (ENSDF, https://www.nndc.bnl.gov/ensdf/).

In [ ]:
import numpy as np

from scipy.constants import physical_constants

from ries.constituents.isotope import Isotope
from ries.constituents.state import GroundState, State
from ries.constituents.element import natural_elements

sn112 = Isotope('112Sn',
        natural_elements['Sn'].isotopes['112Sn'].amu,
        ground_state=GroundState('0^+_1', two_J=0, parity=1),
        excited_states={
            '2^+_1': State(
                '2^+_1', two_J=4, parity=1, 
                excitation_energy=1.25669,
                partial_widths={'0^+_1': physical_constants['reduced Planck constant in eV s'][0]/0.376e-12*1e-6} # Conversion from half life to width
            )
        }
       )

sn112_enrichment = 0.951
# The following calculation for the number of nuclei in the target layer is an approximation. 
# More than 90% of the layer consist of 112Sn,
# but one would actually have to divide by the abundance-weighted average mass of all tin isotopes 
# in the sample.
sn112_1_areal_density = 2.40403e-3/(sn112.amu*physical_constants['atomic mass constant'][0])/(np.pi*0.005e15**2)
sn112_2_areal_density = 2.39414e-3/(sn112.amu*physical_constants['atomic mass constant'][0])/(np.pi*0.005e15**2)


sn116 = Isotope('116Sn',
        natural_elements['Sn'].isotopes['116Sn'].amu,
        ground_state=GroundState('0^+_1', two_J=0, parity=1),
        excited_states={
            '2^+_1': State(
                '2^+_1', two_J=4, parity=1, 
                excitation_energy=1.293560,
                partial_widths={'0^+_1': physical_constants['reduced Planck constant in eV s'][0]/0.374e-12*1e-6} # Conversion from half life to width
            )
        }
       )

sn116_enrichment = 0.978
sn116_1_areal_density = 2.40668e-3/(sn116.amu*physical_constants['atomic mass constant'][0])/(np.pi*0.005e15**2)
sn116_2_areal_density = 2.39617e-3/(sn116.amu*physical_constants['atomic mass constant'][0])/(np.pi*0.005e15**2)

## Cross sections

### Resonances

The cross sections $\sigma_{0^+_1 \to 2^+_1} (E)$ for the excitation of the $2^+_1$ states of the Sn isotopes by photons are Breit-Wigner resonances, since both can be considered isolated resonances.
At standard conditions, the resonances are considerably Doppler-broadened.
Their energy dependence is given by a Voigt profile, which is the convolution of a Breit-Wigner resonance with a Maxwell-Boltzmann distribution of atom velocities.
They are modeled by `Voigt` objects, which use some of the isotope properties defined above.

A Maxwell-Boltzmann distribution can only be used for solid tin, if the temperature in the Maxwell-Boltzmann distribution is replaced by an effective temperature.
To obtain this quantity, the Debye approximation is used here.
The module `ries.resonance.debye_model` provides a dictionary of room-temperature material constants ('Debye temperatures') that are needed to calculate the effective temperature for a given room temperature.
Note that the Debye temperatures in the dictionary are actually valid for elements with a natural composition of isotopes, so this is an approximation.

In [ ]:
from ries.resonance.voigt import Voigt
from ries.resonance.debye_model import effective_temperature_debye_approximation, room_temperature_T_D

room_temperature = 293.

sigma_112 = Voigt(
    initial_state=sn112.ground_state,
    intermediate_state=sn112.excited_states['2^+_1'],
    amu=sn112.amu,
    effective_temperature=effective_temperature_debye_approximation(room_temperature, room_temperature_T_D['Sn']),
)

sigma_116 = Voigt(
    initial_state=sn116.ground_state,
    intermediate_state=sn116.excited_states['2^+_1'],
    amu=sn116.amu,
    effective_temperature=effective_temperature_debye_approximation(room_temperature, room_temperature_T_D['Sn']),
)

Plot the cross section for $^{116}$Sn. Note that both Sn isotopes have similar properties, so the plots for the other isotope would similar.

The `Voigt` class provides a method `equidistant_probability_grid` that partitions the energy range into quantiles of equal size.
Compared to an equidistant partition, this has the advantage that the grid is more dense around the peak and less dense at the tails where the shape does not change much.
For a given number of points, this should increase the precision.
A Voigt distribution is actually defined on the interval $]-\infty, \infty[$, but the energy grid is restricted to a finite symmetric coverage interval here.

In [ ]:
import matplotlib.pyplot as plt

energy_plot = sigma_116.equidistant_probability_grid(0.999, 100) # 99.9% coverage, 200 points at which the cross section is evaluated for plotting.
sigma_plot = sigma_116(energy_plot)

fig, ax = plt.subplots(1,1)
ax.set_xlabel('Energy (MeV)')
ax.set_ylabel(r'$\sigma_{0^+_1 \to 2^+_1}$ ($^{116}$Sn) (fm$^2$)')
ax.plot(energy_plot, sigma_plot, '--', color='crimson')
ax.plot(energy_plot, sigma_plot, '.', color='black')

### Nonresonant processes

Photons will not only undergo resonant reactions in the target, but they can also be Compton-scattered, create an electron-positron pair, ...
The energy dependence of these nonresonant processes can be described by the tabulated x-ray mass attenuation coefficients (XRMAC) $\mu / \rho$ of Hubbell and Seltzer.
The module `ries.nonresonant.xrmac` provides a dictionary of all XRMACs in that database.
While the XRMACs are tabulated as *cross sections per gram*, the ones in `ries.nonresonant.xrmac.xrmac_fm2_per_atom` have already been converted to *cross sections per nucleus* $\sigma_\mathrm{nr} (E)$, i.e. they are directly comparable to $\sigma_{0^+_1 \rightarrow 2^+_1}$.
As with the Debye temperatures, the XRMACs for the two enriched targets are approximated by the one for natural tin.

In [ ]:
from ries.nonresonant.xrmac import xrmac_fm2_per_atom

sigma_nr = xrmac_fm2_per_atom['Sn']

In [ ]:
sigma_nr_plot = sigma_nr(energy_plot) # Plot on the same energy grid as the resonances.

fig, ax = plt.subplots(1,1)
ax.set_xlabel('Energy (MeV)')
ax.set_ylabel(r'$\sigma_\mathrm{nr}$ (fm$^2$)')
ax.plot(energy_plot, sigma_nr_plot, '--', color='crimson')
ax.plot(energy_plot, sigma_nr_plot, '.', color='black')

## Photon flux

The dependence of the energy-differential photon flux $\mathrm{d} \Phi / \mathrm{d} E$ (number of photons incident on the target per energy unit per time interval) on the penetration depth $z$ (defined in units of an areal density, i.e. a length unit times a volume density) into the target is given by the differential equation [1]:

$\frac{\mathrm{d} \Phi}{\mathrm{d} E \mathrm{d} z} (E, z) = -[\sigma_\mathrm{nr} (E) + x(^{A}\mathrm{Sn}) \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^{A}\mathrm{Sn})] \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z)$.

Note that the NRF cross section needs to be multiplied by the abundance of the isotope, to be able to use the areal density from above (the areal density gives the number of nuclei of *any* isotope per unit area).
Since the two excited states of $^{112}$Sn and $^{116}$Sn are sufficiently far apart, only a single resonance is considered here.
In the following, the boundaries between the target layers will be denoted as $z_i$.
The point on the $z$ axis where the beam hits the target first is $z_0$, the point where the target exits the target is $z_4$.

In [ ]:
z_0 = 0.
z_1 = z_0 + sn116_1_areal_density
z_2 = z_1 + sn112_1_areal_density
z_3 = z_2 + sn116_2_areal_density
z_4 = z_3 + sn112_2_areal_density

The solution of the differential equation above inside a single target layer is an exponential decay:

$\frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z) = \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z_i) \exp \{ -[\sigma_\mathrm{nr} (E) + x(^{A}\mathrm{Sn}) \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^{A}\mathrm{Sn})] (z_{i+1} - z_{i}) \}$

The exponential term in this expression is sometimes called the photon flux density.
For a thin target, i.e. 

$\frac{z_{i+1} - z_i}{\sigma_\mathrm{nr} (E) + x(^{A}\mathrm{Sn}) \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^{A}\mathrm{Sn})} \ll 1$,

the variation of the photon flux with the penetration depth is negligible:

$\frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z) \approx \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z_i)$

The functional dependence of the photon flux is implemented as python code below.

In [ ]:
def dPhi_dE(E, z, sigma_nr, sigma_r, abundance=1., dPhi_dE_0=lambda E: 1.):
    r"""Energy- and penetration-depth dependence of the energy-differential photon flux

    It is assumed that the reduction of the photon flux is due to nonresonant scattering by the
    entire target material and absorption by resonances of a single isotope, whose abundance can
    be given as an argument.

    Parameters:
    
    - `E`, float or array_like, energy in MeV.
    - `z`, float or array_like, penetration depth (in units of an areal density) in fm^-2.
    - `sigma_nr` and `sigma_r`, callable, energy-dependent nonresonant (nr) and resonant (r) cross sections.
    - `abundance`, float, abundance of the isotope to which the resonances belong. Should be a number between 0 and 1.
    - `dPhi_dE_0`, callable, energy-differential photon flux at the most upstream point of the target in arbitrary units.
    
    Returns:
    
    - float or array_like, energy-differential photon flux in the same units as `dPhi_dE_0`.
    """
    return dPhi_dE_0(E)*np.exp(-(sigma_nr(E)+abundance*sigma_r(E))*z)

Plot the energy-differential photon flux for different penetration depths into the target.

In [ ]:
dPhi_dE_plot = [dPhi_dE(energy_plot, z, sigma_nr, sigma_116, abundance=sn116_enrichment) for z in np.linspace(0., sn116_1_areal_density, 10)]

fig, ax = plt.subplots(1,1)
ax.set_xlabel('Energy (MeV)')
ax.set_ylabel('$\mathrm{d} \Phi \mathrm{d} E$ (arb. units)')
for dP_dE in dPhi_dE_plot:
    ax.plot(energy_plot, dP_dE)

## Reaction rate

The energy- and penetration-depth differential reaction rate $\mathrm{d} N_\mathrm{NRF} (^A\mathrm{Sn}) / \mathrm{d} E \mathrm{d} z$ in a single target layer is given by the product of the cross section and the energy-differential photon flux:

$\frac{\mathrm{d} N_\mathrm{NRF} (^A\mathrm{Sn})}{\mathrm{d} E \mathrm{d} z} = x(^{A}\mathrm{Sn}) \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^A\mathrm{Sn}) \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z)$

This quantity is sometimes called the resonance absorption density and denoted as $\alpha$.

By integrating the expression above over the energy and the extent of the target layer, one obtains the reaction rate in this layer:

$N_\mathrm{NRF} (^A\mathrm{Sn}) = x(^{A}\mathrm{Sn}) \int_{-\infty}^{\infty} \int_{z_i}^{z_{i+1}} \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^A\mathrm{Sn}) \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z) \mathrm{d} z \mathrm{d} E$

For a thin target, the photon flux is constant inside the target, the the integral evaluates to:

$N_\mathrm{NRF} (^A\mathrm{Sn}) \approx x(^{A}\mathrm{Sn}) \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z_i) \int_{-\infty}^{\infty} \sigma_{0^+_1 \rightarrow 2^+_1} (E; ^A\mathrm{Sn}) \mathrm{d} E = x(^{A}\mathrm{Sn}) \frac{\mathrm{d} \Phi}{\mathrm{d} E} (E, z_i) (z_{i+1} - z_i) I_{0^+_1 \rightarrow 2^+_1} (^A\mathrm{Sn})$,

i.e. the count rate is given by the product of the abundance, the incident photon flux, the areal density, and the energy-integrated cross section $I_{0^+_1 \rightarrow 2^+_1} (^A\mathrm{Sn})$.

The functional dependence of the resonance absorption density is implemented as python code below.

In [ ]:
def dalpha_dE_dz(E, z, sigma_nr, sigma_r, abundance=1., dPhi_dE_0=lambda E: 1.):
    r"""Energy- and penetration-depth differential resonance absorption density

    See also dPhi_DE.

    Parameters:
    
    - `E`, float or array_like, energy in MeV.
    - `z`, float or array_like, penetration depth (in units of an areal density) in fm^-2.
    - `sigma_nr` and `sigma_r`, callable, energy-dependent nonresonant (nr) and resonant (r) cross sections.
    - `abundance`, float, abundance of the isotope to which the resonances belong. Should be a number between 0 and 1.
    - `dPhi_dE_0`, callable, energy-differential photon flux at the most upstream point of the target in arbitrary units.
    
    Returns:
    
    - float or array_like, energy- and penetration-depth differential resonance absorption density in units of fm^2.
    """
    return abundance*sigma_r(E)*dPhi_dE(E, z, sigma_nr, sigma_r, abundance, dPhi_dE_0)

In [ ]:
alpha_plot = [dalpha_dE_dz(energy_plot, z, sigma_nr, sigma_116, sn116_enrichment) for z in np.linspace(0., sn116_1_areal_density, 10)]

fig, ax = plt.subplots(1,1)
for alpha in alpha_plot:
    ax.plot(energy_plot, alpha)